In [2]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
##not working?

In [3]:
# 2/21/2019 - 2/20/2024
df=pd.read_csv("AAPL.csv")
print("Number of rows and columns:", df.shape)
df.head(5)

Number of rows and columns: (1258, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-02-21,42.950001,43.092499,42.575001,42.764999,41.169819,68998800
1,2019-02-22,42.895000,43.250000,42.845001,43.242500,41.629501,75652800
2,2019-02-25,43.540001,43.967499,43.487499,43.557499,41.932755,87493600
3,2019-02-26,43.427502,43.825001,43.292500,43.582500,41.956833,68280800
4,2019-02-27,43.302502,43.750000,43.182499,43.717499,42.086788,111341600


In [4]:
training_set = df.iloc[:1007, 1:2].values
test_set = df.iloc[1007:, 1:2].values


X_train[0]: This is the first 60-day sequence of stock prices, used as the first input to the model. After reshaping, each price within this sequence is now encapsulated within its own array. So, if originally X_train[0] was [price_1, price_2, ..., price_60], after reshaping, it becomes [[price_1], [price_2], ..., [price_60]]. This means X_train[0] is now a 2D array of shape (60, 1), where each day's price is in its own sub-array, matching the expected input shape for many machine learning models, especially RNNs and LSTMs.

X_train[1]: Similarly, this is the second 60-day sequence of stock prices, with each price also encapsulated within its own array. If the original second sequence was [price_61, price_62, ..., price_120], it becomes [[price_61], [price_62], ..., [price_120]] after reshaping. Like X_train[0], X_train[1] is now a 2D array of shape (60, 1).

In [5]:
# Assuming training_set_scaled has been defined as per your previous steps
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, 1007):  # Adjusted to use the full range of your training data
    #day 1 - 60
    X_train.append(training_set_scaled[i-60:i, 0])  # 60 previous days' stock prices
    #day 61 price used for prediction 
    y_train.append(training_set_scaled[i, 0])       # Current day's stock price
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping X_train for LSTM layer input (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [6]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
30/30 [==============================] - 3s 31ms/step - loss: 0.0580
Epoch 2/100
30/30 [==============================] - 1s 32ms/step - loss: 0.0080
Epoch 3/100
30/30 [==============================] - 1s 32ms/step - loss: 0.0081
Epoch 4/100
30/30 [==============================] - 1s 32ms/step - loss: 0.0074
Epoch 5/100
30/30 [==============================] - 1s 33ms/step - loss: 0.0070
Epoch 6/100
30/30 [==============================] - 1s 34ms/step - loss: 0.0061
Epoch 7/100
30/30 [==============================] - 1s 35ms/step - loss: 0.0064
Epoch 8/100
30/30 [==============================] - 1s 36ms/step - loss: 0.0059
Epoch 9/100
30/30 [==============================] - 1s 34ms/step - loss: 0.0054
Epoch 10/100
30/30 [==============================] - 1s 34ms/step - loss: 0.0049
Epoch 11/100
30/30 [==============================] - 1s 34ms/step - loss: 0.0052
Epoch 12/100
30/30 [==============================] - 1s 34ms/step - loss: 0.0054
Epoch 13/100
30/30 [=====

In [7]:
# Getting the predicted stock price of 2017
dataset_train = df.iloc[:1007, 1:2]
dataset_test = df.iloc[1007:, 1:2]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
# Replace 519 with dynamic value based on inputs length
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(251, 60, 1)


In [8]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

8/8 [==============================] - 1s 8ms/step


In [9]:
import plotly.graph_objects as go

# Get the end date
end_date = df.loc[1007:, "Date"].max()

# Create a trace for the real stock price
trace1 = go.Scatter(
    x = df.loc[1007:, "Date"],
    y = dataset_test.values.flatten(),
    mode = 'lines',
    name = 'Real AAPL Stock Price'
)

# Create a trace for the predicted stock price
trace2 = go.Scatter(
    x = df.loc[1007:, "Date"],
    y = predicted_stock_price.flatten(),
    mode = 'lines',
    name = 'Predicted AAPL Stock Price'
)

data = [trace1, trace2]

# Define the layout
layout = go.Layout(
    title = 'AAPL Stock Price Prediction',
    xaxis = dict(title = 'Time'),
    yaxis = dict(title = 'AAPL Stock Price'),
    annotations=[
        dict(
            x=end_date,
            y=0,
            xref='x',
            yref='paper',
            text='End Date: {}'.format(end_date),
            showarrow=False
        )
    ]
)

# Create a Figure and plot it
fig = go.Figure(data=data, layout=layout)
fig.show()

In [20]:
import numpy as np
from pandas.tseries.offsets import DateOffset

# Number of previous days' data used for prediction
n_input = 60

# Get the last `n_input` days from the training data set
last_days = training_set_scaled[-n_input:]

# Reshape to match the input shape of the model
last_days_scaled = np.reshape(last_days, (1, n_input, 1))

# Number of future days to predict
n_future = 30  # e.g. predict the next month

# List to hold the predictions
predictions = []

for _ in range(n_future):
    # Predict the next day
    predicted_value = model.predict(last_days_scaled)
    
    # Reshape predicted_value to match the dimensions of last_days_scaled
    predicted_value = np.reshape(predicted_value, (1, 1, 1))
    
    # Append the predicted value to the list of predictions
    predictions.append(sc.inverse_transform(predicted_value[0, 0, 0].reshape(-1, 1))[0][0])    # Append the predicted value to the last_days_scaled and remove the first value
    last_days_scaled = np.append(last_days_scaled[:, 1:, :], predicted_value, axis=1)

# Create a DataFrame with the future dates and predicted prices
future_dates = pd.date_range(start='2022-03-01', periods=n_future, freq='B')  # Adjust the start date as needed
predicted_prices = np.array(predictions)

prediction_df = pd.DataFrame(data={'Date': future_dates, 'Predicted Close': predicted_prices})

print(prediction_df)

1/1 [==============================] - 0s 9ms/step
         Date  Predicted Close
0  2022-03-01       153.915649
1  2022-03-02       154.278656
2  2022-03-03       154.708206
3  2022-03-04       155.166473
4  2022-03-07       155.620453
5  2022-03-08       156.051697
6  2022-03-09       156.455124
7  2022-03-10       156.830856
8  2022-03-11       157.184082
9  2022-03-14       157.518295
10 2022-03-15       157.837509
11 2022-03-16       158.145401
12 2022-03-17       158.443741
13 2022-03-18       158.734543
14 2022-03-21       159.019043
15 2022-03-22       159.297104
16 2022-03-23       159.570648
17 2022-03-24       159.839645
18 2022-03-25       160.104141
19 2022-03-28       160.363693
20 2022-03-29       160.618637
21 2022-03-30       160.868195
22 2022-03-31       161.112366
23 2022-04-01       161.351791
24 2022-04-04       161.585907
25 2022-04-05       161.815018
26 2022-04-06       162.039200
27 2022-04-07       162.258255
28 2022-04-08       162.472122
29 2022-04-11      

In [21]:
import plotly.graph_objects as go

# Create a trace for the predicted prices
trace1 = go.Scatter(
    x=prediction_df['Date'],
    y=prediction_df['Predicted Close'],
    mode='lines',
    name='Predicted Close'
)

data = [trace1]

layout = go.Layout(
    title="Predicted Stock Prices",
    xaxis={'title': "Date"},
    yaxis={'title': "Close Price"}
)

fig = go.Figure(data=data, layout=layout)

fig.show()